<a href="https://colab.research.google.com/github/animeshdhakal/PyTorchBasics/blob/main/PyTorch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
input_size = 28*28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [4]:
train_data = torchvision.datasets.MNIST(root="./datasets", train=True, download=True, transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root="./datasets", train=False, download=False, transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [14]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, no_classes):
    super().__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, no_classes)

  def forward(self, x):
    out = self.l1(x)
    out = torch.relu(out)
    out = self.l2(out)
    return out

In [15]:
model = NeuralNet(input_size, hidden_size, num_classes)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
num_steps = len(train_loader)

In [29]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    model.train()
    
    outputs = model(images)

    loss = criterion(outputs, labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if (i+1) % 100 == 0:
      print(f"Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{num_steps}, Loss: {loss.item():.5f}")


Epoch: 1/2, Step: 100/600, Loss: 0.10501
Epoch: 1/2, Step: 200/600, Loss: 0.07399
Epoch: 1/2, Step: 300/600, Loss: 0.11200
Epoch: 1/2, Step: 400/600, Loss: 0.06513
Epoch: 1/2, Step: 500/600, Loss: 0.10713
Epoch: 1/2, Step: 600/600, Loss: 0.07194
Epoch: 2/2, Step: 100/600, Loss: 0.07464
Epoch: 2/2, Step: 200/600, Loss: 0.04218
Epoch: 2/2, Step: 300/600, Loss: 0.02185
Epoch: 2/2, Step: 400/600, Loss: 0.07058
Epoch: 2/2, Step: 500/600, Loss: 0.22826
Epoch: 2/2, Step: 600/600, Loss: 0.11547


In [30]:
with torch.inference_mode():
  no_correct = 0
  no_samples = 0

  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    outputs = model(images)

    _, predictions = torch.max(outputs, 1)

    no_correct += (predictions == labels).sum().item()

    no_samples += images.shape[0]

  acc = (no_correct / no_samples) * 100.0

  print(f"Total Correct Predictions: {no_correct}/{no_samples} Accuracy: {acc}")

Total Correct Predictions: 9705/10000 Accuracy: 97.05
